# Background sampling

This notebooks aims to create a sampling of background points from coastal areas, and an "other" class sampling from other types of habitats that likely do not coexist as neighbours to the habitats of interest.

Steps:

1. Get sub-tiles with no habitat observations. Record these as "Unknown" areas to be used for prediction only. &#x2713;
2. For all habitat points, compute which sub-tiles they belong to. &#x2713;
3. For each UTM tile, compute coastal buffer from Idepix land mask. &#x2713;
4. For each sub-tile computed in step 3, computer cluster envelopes. &#x2713;
5. Sample background points from outside cluster envelopes, and "other" points from other habitat classes. &#x2713;

## Setup

Files and system

In [ ]:
import os
import json

Arrays

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd

Geometry

In [ ]:
from shapely.geometry import Point, MultiPoint

Raster operations

In [ ]:
from scipy.ndimage import binary_dilation

In [ ]:
import rasterio
import rioxarray
from rasterio.features import rasterize

Plotting

In [ ]:
import matplotlib.pyplot as plt

Directories

In [ ]:
DATA_DIR = '../data/'
FIGURES_DIR = '../saved_figures/'

Habitat data

In [ ]:
habitats_shp = os.path.join(DATA_DIR, 'OSPARHabitats2022_Points_clustered/OSPAR2022Points.shp')
habitats_gdf = gpd.read_file(habitats_shp)

Sub-tile grid

In [ ]:
with open(os.path.join(DATA_DIR, 'tile_buffer_wkt.json'), 'r') as f:
    tiles_wkt_dict = json.load(f)

## Get sub-tiles

In [ ]:
habitats_gdf['subtile_index'] = None

In [ ]:
unknown_subtiles = dict()

In [ ]:
# for tileId in tiles_wkt_dict:
#     habitats_sub = habitats_gdf[habitats_gdf['tileId'] == tileId]
#     subtiles = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkt(tiles_wkt_dict[tileId]), crs="EPSG:4326")

#     # Find polygon that contains each point
#     matches = [None]*len(habitats_sub)
#     for j, (i, row) in enumerate(habitats_sub.iterrows()):
#         point = row.geometry
#         matching_idx = subtiles[subtiles.contains(point)].index

#         if not matching_idx.empty:
#             # habitats_gdf.at[i, 'subtile_index'] = matching_idx[0]
#             matches[j] = matching_idx[0]
#         else:
#             print(f"No polygon contains point {point}")

#     # Get sub-tile indices containing no points
#     used_subtile_indices = set(matches) - {None}
#     all_subtile_indices = set(subtiles.index)
#     unknown_subtiles[tileId] = all_subtile_indices - used_subtile_indices

#     # Update subtile_index column
#     habitats_gdf.loc[habitats_sub.index, 'subtile_index'] = matches

More efficient method

In [ ]:
for tileId in tiles_wkt_dict:
    habitats_sub = habitats_gdf[habitats_gdf['tileId'] == tileId].copy()
    subtiles = gpd.GeoDataFrame(
        geometry=gpd.GeoSeries.from_wkt(tiles_wkt_dict[tileId]), crs="EPSG:4326"
    )
    subtiles = subtiles.reset_index().rename(columns={'index': 'subtile_index'})

    # Perform spatial join: keep only points that are inside polygons
    joined = gpd.sjoin(habitats_sub, subtiles, how='left', predicate='contains')

    # Update the main DataFrame with results from spatial join
    habitats_gdf.loc[joined.index, 'subtile_index'] = joined['subtile_index'].values

    # Find subtiles with no points
    used_subtiles = set(joined['subtile_index'].dropna())
    all_subtiles = set(subtiles['subtile_index'])
    unknown_subtiles[tileId] = list(all_subtiles - used_subtiles)


Save habitat data-frame and unknown subtiles.

In [ ]:
habitats_gdf.to_file(habitats_shp)

In [ ]:
with open(os.path.join(DATA_DIR, 'unknown_subtiles.json'), 'w') as f:
    json.dump(unknown_subtiles, f)

## Coastal buffer

In [ ]:
def idepix_coastal_buffer(idepix_ds, buffer_pixels=5):
    land_mask = idepix_ds['IDEPIX_LAND'].astype(bool)

    # Create buffered land mask
    buffered_land = binary_dilation(land_mask, iterations=buffer_pixels)

    # Coastal buffer = buffered land minus original land
    coastal_buffer = buffered_land & ~land_mask

    coastal_buffer_da = xr.DataArray(
        coastal_buffer,
        coords=idepix_ds['IDEPIX_CLEAR_WATER'].coords,
        dims=idepix_ds['IDEPIX_CLEAR_WATER'].dims,
        name='coastal_buffer'
    )

    return coastal_buffer_da

Example

In [ ]:
idepix_df = os.path.join(DATA_DIR, f's2_processed/{tileId}/idepix-{tileId}.nc')
idepix_ds = xr.open_dataset(idepix_df)

In [ ]:
coastal_buffer_da = idepix_coastal_buffer(idepix_ds).astype(bool)

Save

In [ ]:
# coastal_buffer_da.to_file(...

## Cluster envelopes

In [ ]:
def cluster_convex_hulls(gdf, template_da, burn_value=1):
    hull_geoms = []

    for cluster_id in gdf['Cluster'].unique():
        cluster_points = gdf.loc[gdf['Cluster'] == cluster_id, 'geometry']
        multipoint = MultiPoint(cluster_points.to_list())
        hull = multipoint.convex_hull
        hull_geoms.append((hull, burn_value))

    # Get transform and shape from template raster
    transform = template_da.rio.transform()
    out_shape = template_da.shape

    # Rasterize convex hulls
    mask_array = rasterize(
        hull_geoms,
        out_shape=out_shape,
        transform=transform,
        fill=0,
        dtype='uint8'
    )

    # Wrap in a DataArray
    mask_da = xr.DataArray(
        mask_array,
        coords=template_da.coords,
        dims=template_da.dims,
        name='cluster_convex_hull_mask'
    )

    return mask_da

Example

In [ ]:
cluster_mask = cluster_convex_hulls(gdf, idepix_ds['IDEPIX_CLEAR_WATER']).astype(bool)

In [ ]:
background_mask = coastal_buffer_bin & ~cluster_mask_bin
background_mask.name = 'background_mask'

## Sample background

In [ ]:
def sample_pixels_from_mask(mask_da, n_samples, seed=None):
    valid_y, valid_x = np.where(mask_da.samples == 1)

    if len(valid_x) < n_samples:
        raise ValueError(f"Requested {n_samples} samples, but only {len(valid_x)} valid pixels available.")

    rng = np.random.default_rng(seed)
    sampled_indices = rng.choice(len(valid_x), size=n_samples, replace=False)

    rows = valid_y[sampled_indices]
    cols = valid_x[sampled_indices]

    return rows, cols

Example

In [ ]:
sampled_points = sample_pixels_from_mask(background_mask, n_samples=100, seed=42)

Save

In [ ]:
# sampled_points_gdf = ...

In [ ]:
# sampled_points_gdf.to_file(...

## Sample other points

In [ ]:
def sample_point_pixels(gdf, coords_tree, n_samples, seed=None):
    if n_samples > len(gdf):
        raise ValueError("Requested more samples than available in gdf.")
    
    rng = np.random.default_rng(seed)

    # sample randomly
    sampled_indices = rng.choice(len(gdf), size=n_samples, replace=False)

    # find pixel locations
    geoms = gdf.loc[sampled_indices, 'geometry']
    _, pixel_locs = coords_tree.query(np.column_stack((geoms.values.x, geoms.values.y)))

    return pixel_locs

Example

In [ ]:
hab_of_interest = ['Zostera beds', 'Kelp forests']

In [ ]:
other_gdf = gdf[~gdf['HabType'].isin(hab_of_interest)]

In [ ]:
tile_gdf = other_gdf[other_gdf['tileId'] == tileId]

In [ ]:
subtile_gdf = tile_gdf[tile_gdf['subtile_index'] == subtile_idx]

In [ ]:
latitudes = template_da['lat'].values.ravel()
longitudes = template_da['lon'].values.ravel()

coords_tree = cKDTree(np.column_stack((latitudes, longitudes)))

other_samples = sample_point_pixels(subtile_gdf, coords_tree, 100, seed=42)

In [ ]:
# for tileId in ...
# for subtile_idx in ...

## Loop over tiles and subtiles

In [ ]:
for tileId in tiles_wkt_dict:
    idepix_df = os.path.join(DATA_DIR, f's2_processed/{tileId}/idepix-{tileId}.nc')
    idepix_ds = xr.open_dataset(idepix_df)
    coastal_buffer_da = idepix_coastal_buffer(idepix_ds).astype(bool)

    tile_habitats_gdf = habitats_gdf[habitats_gdf['utm_tile'] == tileId]

    